In [25]:
using QuadGK
using Interpolations

In [54]:
f(x) = 1/exp(x)
g(x) = 2*x

g (generic function with 1 method)

In [39]:
a = [[1, 0], [2, 3]]

1

In [40]:
dpdz = [[0, 1, 2,
    3, 2, 1,
    0, 1, 2,
    3, 2], [0, 0.1, 0.2,
    0.3, 0.4, 0.5,
    0.6, 0.7, 0.8,
    0.9, 1]]

2-element Array{Array{Float64,1},1}:
 [0.0, 1.0, 2.0, 3.0, 2.0, 1.0, 0.0, 1.0, 2.0, 3.0, 2.0]
 [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [46]:
function get_pz(dpdz)
    p = dpdz[1]
    z = dpdz[2]
    pz = LinearInterpolation(z, p)
    return pz
end

get_pz (generic function with 1 method)

In [47]:
pz = get_pz(dpdz)

11-element extrapolate(interpolate((::Array{Float64,1},), ::Array{Float64,1}, Gridded(Linear())), Throw()) with element type Float64:
 0.0
 1.0
 2.0
 3.0
 2.0
 1.0
 0.0
 1.0
 2.0
 3.0
 2.0

In [50]:
pz(0.27)

2.7

In [56]:
quadgk(f)

MethodError: [91mMethodError: no method matching quadgk(::typeof(f))[39m
[91m[0mClosest candidates are:[39m
[91m[0m  quadgk(::Any, [91m::T[39m, [91m::T[39m, [91m::T...[39m; atol, rtol, maxevals, order, norm) where T at /home/jaimerz/.julia/packages/QuadGK/czbUH/src/adapt.jl:177[39m
[91m[0m  quadgk(::Any, [91m::Any[39m, [91m::Any[39m, [91m::Any...[39m; kws...) at /home/jaimerz/.julia/packages/QuadGK/czbUH/src/adapt.jl:173[39m

In [22]:
function integrator(a, b)
    pz = get_pz()LinearInterpolation(data[:,1], data[:,2])
    g(x) = 1/exp(2*x)
    return quadgk(g, a, b)
end
    

integrator (generic function with 2 methods)

In [24]:
integrator(1, 0)

(-0.4323323583816937, 4.996003610813204e-16)

In [21]:
integrator()

(0.4323323583816937, 4.440892098500626e-16)

In [52]:
function w(z, f, g)
    return f(z)*g(z) 
end

w (generic function with 1 method)

In [55]:
w(1, f, g)

0.7357588823428847